# Lecture: Network Analysis
This notebook is an introduction to network analysis using the newest version 2.1 of the <a href='https://networkx.github.io/'>networkx</a> library. <font face='Courier'>networkx</font> is not suited for large networks, but it's part of the standard Anaconda installation. For large networks, consider <a href='https://graph-tool.skewed.de/'>graph-tool</a> or <a href='http://igraph.org/python/'>igraph</a>. Consult <a href='https://doi.org/10.1177/2059799115622763'>this paper</a> for a comparison of these packages and why Python is good for scientific computing. If you're looking for another introduction into <font face='Courier'>networkx</font>, you can try <a href='https://networkx.github.io/documentation/stable/tutorial.html'>this tutorial</a>.

## Imports and Settings

In [ ]:
%matplotlib inline

In [ ]:
import warnings
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite

In [ ]:
warnings.filterwarnings('ignore')

## Graph Construction
Terminology: Graphs consist of nodes (entities like persons or hashtags) and edges that connect nodes. By definition, an edge is defined by a pair of nodes. Both nodes and edges can have properties, and we will be using size and color for nodes and width and color for edges. If edges are directed, the graph is called a <font face='Courier'>DiGraph</font>.

Create an empty graph:

In [ ]:
G = nx.Graph()

To create a directed graph:

In [ ]:
D = nx.DiGraph()

### Populate Graph Using Labels
Add nodes:

In [ ]:
labels = (['Alice', 'Bob', 'Carol'])
G.add_nodes_from(labels)

In [ ]:
G.nodes()

Alternatively:

In [ ]:
for label in labels:
    G.add_node(label)

In [ ]:
G.nodes()

Add edges:

In [ ]:
edges_label = [['Alice', 'Bob'], ['Bob', 'Carol'], ['Alice', 'Carol']]
G.add_edges_from(edges_label)

In [ ]:
G.edges()

Alternatively:

In [ ]:
for edge in range(0, len(edges_label)):
    G.add_edge(edges_label[edge][0], edges_label[edge][1])

In [ ]:
G.edges()

Obtain graph with integer labels:

In [ ]:
H = nx.convert_node_labels_to_integers(G)

In [ ]:
H.nodes()

In [ ]:
H.edges()

### Populate Graph Using Integers

In [ ]:
G = nx.Graph()

In [ ]:
nodes = ([0, 1, 2])
G.add_nodes_from(nodes)

In [ ]:
G.nodes()

In [ ]:
edges = [[0, 1], [0, 2], [1, 2]]
G.add_edges_from(edges)

In [ ]:
G.edges()

Add labels to nodes:

In [ ]:
for i in range(0, len(labels)):
    G.nodes[i]['label'] = labels[i]

In [ ]:
G.nodes(data=True)

In [ ]:
G.edges(data=True)

Get neighbors of selected nodes:

In [ ]:
[n for n in G.neighbors(0)]

## Plot Graph

In [ ]:
plt.figure(figsize=(2, 2))
nx.draw_circular(G, with_labels=True)

In [ ]:
plt.figure(figsize=(2, 2))
nx.draw_spring(G, with_labels=True)

In [ ]:
D = nx.DiGraph()
D.add_edges_from(edges)

In [ ]:
plt.figure(figsize=(2, 2))
nx.draw_spring(D, with_labels=True)

## Bipartite Networks
Bipartite networks consist of two types of nodes -- like agents being influenced by social facts --, and there are no edges among nodes of the same type.

![bipartite](img/bipartite.png)

Load and visualize bipartite data:

In [ ]:
edgelist = pd.read_csv('data/bipartite.txt', header='infer', delimiter='\t', encoding='utf-8')
edgelist

Use edgelist to create graph:

In [ ]:
#G = nx.from_pandas_dataframe(edgelist, source='agent', target='fact') # networkx 2.0
G = nx.Graph()
G.add_edges_from(edgelist.values)

Manually set node positions:

In [ ]:
pos_bipartite = {'a1': np.array([-0.33333333,  0.66666667]),
 'a2': np.array([-0.33333333,  0.33333333]),
 'a3': np.array([-0.33333333,  0.        ]),
 'a4': np.array([-0.33333333, -0.33333333]),
 'a5': np.array([-0.33333333, -0.66666667]),
 'f1': np.array([ 0.33333333,  0.80000000]),
 'f2': np.array([ 0.33333333,  0.60000000]),
 'f3': np.array([ 0.33333333,  0.40000000]),
 'f4': np.array([ 0.33333333,  0.20000000]),
 'f5': np.array([ 0.33333333,  0.        ]),
 'f6': np.array([ 0.33333333, -0.20000000]),
 'f7': np.array([ 0.33333333, -0.40000000]),
 'f8': np.array([ 0.33333333, -0.60000000]),
 'f9': np.array([ 0.33333333, -0.80000000])}

In [ ]:
G.nodes()

In [ ]:
plt.figure(figsize = (3, 3))
nx.draw(G, pos=pos_bipartite, with_labels=True, node_color='w')

Bipartite networks can be folded into unipartite networks. To do that, the two node sets of the bipartite matrix must be "colored", *i.e.* the two partitions must be identified:

In [ ]:
#agents, facts = bipartite.sets(G)

In [ ]:
nx.is_connected(G)

In [ ]:
agents, facts = set(edgelist['agent']), set(edgelist['fact'])

In [ ]:
agents, facts

Create the agent co-fact graph. An edge gives the number of facts a pair of agents is co-influenced by:

In [ ]:
G_agents = bipartite.weighted_projected_graph(G, agents)
G_agents.edges(data=True)

Extract edge weights:

In [ ]:
co_facts = [w['weight'] for (u, v, w) in G_agents.edges(data=True)]
co_facts

In [ ]:
plt.figure(figsize=(3, 3))
nx.draw_circular(G_agents, with_labels=True, node_color='w', width=co_facts)

Create the fact co-agent graph (an edge gives the number of agents that a pair of facts co-influence) and extract edge weights:

In [ ]:
G_facts = bipartite.weighted_projected_graph(G, facts)
co_agents = [w['weight'] for (u, v, w) in G_facts.edges(data=True)]
plt.figure(figsize = (3, 3))
nx.draw_circular(G_facts, with_labels=True, node_color='w', width=co_agents)

### Duality of Groups and Link Communities

Assume that a1 and a2 are in group 0; a3, a4, and a5 are in group 1 in the agent network. In practice, such a partition would likely stem from a clustering algorithm.

In [ ]:
agents_groups = pd.read_csv('data/bipartite_agents_groups.txt', header='infer', delimiter='\t', encoding='utf-8')
agents_groups

Create a set of agents for each group:

In [ ]:
agents_group0 = set(agents_groups[agents_groups['group'] == 0]['agent'])
agents_group1 = set(agents_groups[agents_groups['group'] == 1]['agent'])
agents_group0, agents_group1

Transform dataframe into dictionary:

In [ ]:
agents_groups_dict = agents_groups.set_index('agent')['group'].to_dict()
agents_groups_dict

Add colors to nodes by iterating through the dictionary:

In [ ]:
G_agents.nodes(data=True)

In [ ]:
for key, value in agents_groups_dict.items():
    if value == 0:
        G_agents.nodes[key]['color'] = 'r'
    if value == 1:
        G_agents.nodes[key]['color'] = 'b'

In [ ]:
G_agents.nodes(data=True)

Store colors in a list:

In [ ]:
agents_color = [c['color'] for (u, c) in G_agents.nodes(data=True)]
agents_color

In [ ]:
plt.figure(figsize = (3, 3))
nx.draw_circular(G_agents, with_labels=True, node_color=agents_color, width=co_facts)

Using this information, we can construct a graph of facts where now the edges are colored, indicating the co-influence of facts in a group. The way to go is to create one graph for each group and then use the edges of those *layers* to populate a multigraph.

First, extract an edgelist for each group:

In [ ]:
edgelist_group0 = edgelist[edgelist['agent'].isin(agents_group0)]
edgelist_group1 = edgelist[edgelist['agent'].isin(agents_group1)]
edgelist_group0, edgelist_group1

Second, create a bipartite graph for each group:

In [ ]:
G_group0 = nx.Graph()
G_group0.add_edges_from(edgelist_group0.values)
G_group1 = nx.Graph()
G_group1.add_edges_from(edgelist_group1.values)

Third, create a set of facts for each group -- note the node and edge overlap:

In [ ]:
facts_group0, facts_group1 = set(edgelist_group0['fact']), set(edgelist_group1['fact'])
facts_group0, facts_group1

Fourth, fold each biartite graph into a fact co-agent graph:

In [ ]:
G_facts_group0 = bipartite.weighted_projected_graph(G_group0, facts_group0)
G_facts_group1 = bipartite.weighted_projected_graph(G_group1, facts_group1)

Fifth, extract edge weights:

In [ ]:
co_agents_group0 = [w['weight'] for (u, v, w) in G_facts_group0.edges(data=True)]
co_agents_group1 = [w['weight'] for (u, v, w) in G_facts_group1.edges(data=True)]
co_agents_group0, co_agents_group1

Sixth, create a multigraph (where there can be multiple edges among the same node pair) and populate it using the edges from the two group layers:

In [ ]:
G_facts_groups = nx.MultiGraph()
G_facts_groups.add_nodes_from(G_facts.nodes())
G_facts_groups.add_edges_from(G_facts_group0.edges(data=True), color='r')
G_facts_groups.add_edges_from(G_facts_group1.edges(data=True), color='b')
G_facts_groups.edges(data=True)

Seventh, extract colors and edge weights for the multigraph. Nodes are not colored because they can influence agents in both groups (they are neutral regarding groups):

In [ ]:
co_agents_colors = [c['color'] for (u, v, c) in G_facts_groups.edges(data=True)]
co_agents_groups = [w['weight'] for (u, v, w) in G_facts_groups.edges(data=True)]

Finally, plot the multigraph:

In [ ]:
plt.figure(figsize=(3, 3))
nx.draw_circular(G_facts_groups, with_labels=True, node_color='w', edge_color=co_agents_colors, width=co_agents_groups)

Nodes, edges, and labels can also be plotted step by step. This is advantageous if we want to use transparecy solely on the edges:

In [ ]:
pos = nx.circular_layout(G_facts_groups)

In [ ]:
plt.figure(figsize=(3, 3))
nx.draw_networkx_nodes(G_facts_groups, pos=pos, node_color='w')
nx.draw_networkx_labels(G_facts, pos=pos)
nx.draw_networkx_edges(G_facts_groups, pos=pos, edge_color=co_agents_colors, width=co_agents_groups, alpha=0.5)
plt.axis('off')

## How Politicians Talk On Twitter (Bundestagswahl 2013)
Now let's work with empirical data: tweets by German politicians before and shortly after the federal election of 2013. A version of this data was analyzed in <a href='https://www.aaai.org/ocs/index.php/ICWSM/ICWSM14/paper/viewPaper/8069'>this</a> paper.

### Structure of Network Data

Network data is practically stored as a relational database consisting of entities and their relationships. There are two types of entities, politicians (agents) and hashtags (facts):

In [ ]:
politicians = pd.read_csv('data/icwsm14_politicians.txt', delimiter='\t', encoding='utf-8')
politicians.head()

In [ ]:
hashtags = pd.read_csv('data/icwsm14_hashtags.txt', delimiter='\t', encoding='utf-8')
hashtags.head()

There are three types of relationships, mentioning (of politicians by politicians), retweeting (of politicians by politicians), and tagging (of tweets by politicians):

In [ ]:
edgelist_mentioning = pd.read_csv('data/icwsm14_mentioning.txt', delimiter='\t', encoding='utf-8')
edgelist_mentioning = edgelist_mentioning[edgelist_mentioning['agent_id'] != edgelist_mentioning['fact_id']]
edgelist_mentioning.head()

In [ ]:
edgelist_retweeting = pd.read_csv('data/icwsm14_retweeting.txt', delimiter='\t', encoding='utf-8')
edgelist_retweeting = edgelist_retweeting[edgelist_retweeting['agent_id'] != edgelist_retweeting['fact_id']]
edgelist_retweeting.head()

In [ ]:
edgelist_tagging = pd.read_csv('data/icwsm14_tagging.txt', delimiter='\t', encoding='utf-8')
edgelist_tagging.head()

### Graph Visualization
A good way to start an analysis is to get a visual impression of the data. Let's start with the directed unipartite graph of who mentions who. Tune <font face='Courier'>k</font> (the optimal distance between nodes) until the result looks satisfactory:

In [ ]:
#D_mentioning = nx.from_pandas_dataframe(edgelist_mentioning, source='agent_id', target='fact_id', create_using=nx.DiGraph()) # networkx 2.0
D_mentioning = nx.DiGraph()
D_mentioning.add_edges_from(edgelist_mentioning[['agent_id', 'fact_id']].values)
pos_mentioning = nx.spring_layout(D_mentioning, k=0.5)
plt.figure(figsize=(6, 6))
nx.draw(D_mentioning, pos=pos_mentioning)

In [ ]:
nx.number_of_nodes(D_mentioning), nx.number_of_edges(D_mentioning)

Is the graph weakly connected (it is if it contains a directed path from <font face='Courier'>u</font> to <font face='Courier'>v</font> or a directed path from <font face='Courier'>v</font> to <font face='Courier'>u</font> for every pair of vertices <font face='Courier'>u</font>, <font face='Courier'>v</font>)?

In [ ]:
nx.is_weakly_connected(D_mentioning)

How many weakly connected components are there?

In [ ]:
nx.number_weakly_connected_components(D_mentioning)

Extract the largest weakly connected component:

In [ ]:
D_mentioning_giant = max(nx.weakly_connected_component_subgraphs(D_mentioning), key=len)
pos_mentioning_giant = nx.spring_layout(D_mentioning, k=0.5)
plt.figure(figsize=(6, 6))
nx.draw(D_mentioning_giant, pos=pos_mentioning_giant)

In [ ]:
nx.number_of_nodes(D_mentioning_giant), nx.number_of_edges(D_mentioning_giant)

We can do better than this!

First, since we've used a simple <font face='Courier'>nx.DiGraph()</font> structure edge weights are neglected. We can include them by grouping the initial dataframe on source and target nodes:

In [ ]:
edgelist_mentioning_weighted = edgelist_mentioning.groupby(['agent_id', 'fact_id']).size().to_frame('weight').reset_index()
edgelist_mentioning_weighted.head()

In [ ]:
D_mentioning = nx.DiGraph()

In [ ]:
#D_mentioning = nx.from_pandas_dataframe(edgelist_mentioning_weighted, source='agent_id', target='fact_id', edge_attr='weight', create_using=nx.DiGraph()) # networkx 2.0
D_mentioning = nx.DiGraph()
D_mentioning.add_weighted_edges_from(edgelist_mentioning_weighted.values)

In [ ]:
mentioning_weight = [w['weight'] for (u, v, w) in D_mentioning.edges(data=True)]
mentioning_weight = [w/20 for w in mentioning_weight]

In [ ]:
plt.figure(figsize=(6, 6))
nx.draw(D_mentioning, pos=pos_mentioning, width=mentioning_weight)

Second, size nodes by the number of times the corresponding politician has been mentioned:

In [ ]:
mentioning_inweight = dict(D_mentioning.in_degree(weight='weight'))

In [ ]:
for key, value in mentioning_inweight.items():
    D_mentioning.nodes[key]['inweight'] = value

In [ ]:
mentioning_inweight = [w['inweight'] for (u, w) in D_mentioning.nodes(data=True)]
mentioning_inweight = [w/1 for w in mentioning_inweight]

In [ ]:
plt.figure(figsize=(6, 6))
nx.draw(D_mentioning, pos=pos_mentioning, node_size=mentioning_inweight, width=mentioning_weight)

Third, each politician belongs to a political party. Represent each by a color:

In [ ]:
politicians_parties_mentioning = politicians[politicians['agent_id'].isin(set(D_mentioning.nodes))][['agent_id', 'party']]
politicians_parties_mentioning.head()

In [ ]:
politicians_parties_mentioning_dict = politicians_parties_mentioning.set_index('agent_id')['party'].to_dict()

In [ ]:
for key, value in politicians_parties_mentioning_dict.items():
    if value == 'CDU':
        D_mentioning.nodes[key]['color'] = 'black'
    if value == 'CSU':
        D_mentioning.nodes[key]['color'] = 'black'
    if value == 'FDP':
        D_mentioning.nodes[key]['color'] = 'yellow'
    if value == 'Die Grünen':
        D_mentioning.nodes[key]['color'] = 'green'
    if value == 'Die Linke':
        D_mentioning.nodes[key]['color'] = 'darkviolet'
    if value == 'Piratenpartei':
        D_mentioning.nodes[key]['color'] = 'orange'
    if value == 'SPD':
        D_mentioning.nodes[key]['color'] = 'red'

In [ ]:
politicians_color_mentioning = [c['color'] for (u, c) in D_mentioning.nodes(data=True)]

In [ ]:
plt.figure(figsize=(6, 6))
nx.draw(D_mentioning, pos=pos_mentioning, node_size=mentioning_inweight, node_color=politicians_color_mentioning, width=mentioning_weight)

Now that we see more, increase the number of iterations (from a default value of 50) for the spring embedder to converge:

In [ ]:
pos_mentioning = nx.spring_layout(D_mentioning, k=1, iterations=200)
plt.figure(figsize=(6, 6))
nx.draw(D_mentioning, pos=pos_mentioning, node_size=mentioning_inweight, node_color=politicians_color_mentioning, width=mentioning_weight)

### Party Mixing Pattern
Eye inspection of the network shows that politicians strongly mention fellow party members. Newman's assortativity measure (in <a href='https://doi.org/10.1103/PhysRevE.67.026126'>this</a> paper) can be used to quantify this:

In [ ]:
nx.attribute_assortativity_coefficient(D_mentioning, attribute='color')

### Distributions
Eye inspection also shows that there are large differences in attention. Who are those highly mentioned politicians? Add real names to nodes and list them along with inweights:

In [ ]:
politicians_names = politicians[politicians['agent_id'].isin(set(D_mentioning.nodes))][['agent_id', 'agent']]
politicians_names_dict = politicians_names.set_index('agent_id')['agent'].to_dict()

In [ ]:
for key, value in politicians_names_dict.items():
    D_mentioning.nodes[key]['agent'] = value

In [ ]:
pd.DataFrame({
    'agent': list(nx.get_node_attributes(D_mentioning, 'agent').values()),
    'inweight': list(nx.get_node_attributes(D_mentioning, 'inweight').values())
}).sort_values(by='inweight', ascending=False)

A primer on scale-free networks:

<tr><td><img src="img/distr.png" alt="distribution"/></td><td><img src="img/sampl.png" alt="sampling"/></td></tr>

We are used to normal distributions. If the degrees $k$ of a graph are normally distributed, the network has a typical scale described by the mean and standard deviation of the normal distribution (left figure: Gaussian).

<a href='http://doi.org/10.1126/science.286.5439.509'>Scale-free networks</a> are a paradigmatic class of graphs in network science. A network is scale-free if its degree distribution resembles a power law:

$p(k)\propto k^{-\alpha}$

Power laws show as straight lines on log-log scales, where the exponent $\alpha$ determines the slope (left figure: power law). 

In the depicted case, $\alpha\approx2.8$. Networks with exponents $\alpha\leq3$ theoretically have infinite variance, *i.e.* practically samples converge slowly to the empirical value.

We can also use the weighted indegree to study if the mentioning network is scale-free:

In [ ]:
w = [w for w in mentioning_inweight if w > 0]

In [ ]:
import powerlaw as pl

In [ ]:
pl.plot_pdf(w, marker='o', ls='', linear_bins=True) # explore setting linear_bins=False
plt.xlabel('$w$')
plt.ylabel('$p(w)$')

There are five candidate functions to describe distributions like this:

- exponential (like the tail of a normal distribution)
- stretched exponential (an exponential with a power-law stretch)
- lognormal
- power law
- truncated power law (a power law with an exponential upper cutoff)

In [ ]:
function = ['exponential', 'stretched_exponential', 'lognormal', 'power_law', 'truncated_power_law']

Fit these functions to the data (using maximum likelihood estimation described here):

In [ ]:
w_fit = pl.Fit(w, discrete=True, xmin=1) # explore not setting xmin

In [ ]:
print('xmin:', w_fit.xmin)
print('alpha:', round(w_fit.alpha, 2))

Print the Kolmogorov-Smirnov goodness of fit:

In [ ]:
print('exponential:', round(w_fit.exponential.D, 2))
print('stretched_exponential:', round(w_fit.stretched_exponential.D, 2))
print('lognormal:', round(w_fit.lognormal.D, 2))
print('power_law:', round(w_fit.power_law.D, 2))
print('truncated_power_law:', round(w_fit.truncated_power_law.D, 2))

Add fits to plot:

In [ ]:
w_fit.plot_pdf(linear_bins=False, marker='o', ls='', label='data')
w_fit.exponential.plot_pdf(label='exponential')
w_fit.stretched_exponential.plot_pdf(label='stretched_exponential')
w_fit.lognormal.plot_pdf(label='lognormal')
w_fit.power_law.plot_pdf(label='power_law')
w_fit.truncated_power_law.plot_pdf(label='truncated_power_law')
plt.xlabel('$w$')
plt.ylabel('$p(w)$')
plt.legend()

Assess which of two functions is a more plausible fit. The first score, $R$, is the loglikelihood ratio between the two functions. It is positive (negative) when the first (second) function is more likely. The second score, $p$, gives the significance of the difference.

In [ ]:
w_fit.distribution_compare('power_law', 'stretched_exponential')

In [ ]:
w_fit.distribution_compare(function[3], function[1])

Compare functions systematically by comparing all possible pairs:

In [ ]:
w_fit_R = np.zeros((5, 5), dtype=float)
w_fit_p = np.zeros((5, 5), dtype=float)

In [ ]:
for i in range(0, 5):
    for j in range(0, 5):
        R, p = w_fit.distribution_compare(function[i], function[j])
        w_fit_R[i, j] = R
        w_fit_p[i, j] = p

In [ ]:
pd.DataFrame(w_fit_R, index=function, columns=function)

In [ ]:
pd.DataFrame(w_fit_p, index=function, columns=function)

### The Undirected Hidden Network
Often attention in tweets is not reciprocated. Agents who are mentioned by many don't necessarily mention those many agents. Huberman *et al.* have proposed in <a href='http://www.firstmonday.dk/ojs/index.php/fm/article/view/2317/2063'>this</a> paper that the "hidden network" of reciprocal ties resembles a social network where a tie indicates mutual awareness.

Get the indegrees (number of mentioners) for the directed graph:

In [ ]:
mentioning_indegree = dict(D_mentioning.in_degree())

In [ ]:
for key, value in mentioning_indegree.items():
    D_mentioning.nodes[key]['indegree'] = value

In [ ]:
mentioning_indegree = [k['indegree'] for (u, k) in D_mentioning.nodes(data=True)]

Create an undirected graph and get the degrees (number of mentioned mentioners):

In [ ]:
G_mentioning = D_mentioning.to_undirected(reciprocal=True)

In [ ]:
mentioning_degree = dict(G_mentioning.degree())

In [ ]:
for key, value in mentioning_degree.items():
    G_mentioning.nodes[key]['degree'] = value

In [ ]:
mentioning_degree = [k['degree'] for (u, k) in G_mentioning.nodes(data=True)]

Store information in a dataframe and do a scatterplot:

In [ ]:
politicians_mentioners = pd.DataFrame({
    'agent': list(nx.get_node_attributes(G_mentioning, 'agent').values()),
    'indegree': list(nx.get_node_attributes(G_mentioning, 'indegree').values()),
    'degree': list(nx.get_node_attributes(G_mentioning, 'degree').values())
})
politicians_mentioners = politicians_mentioners[(politicians_mentioners['degree'] > 0) & (politicians_mentioners['indegree'] > 0)]
politicians_mentioners.head()

In [ ]:
plt.scatter(politicians_mentioners['indegree'], politicians_mentioners['degree'])
plt.xlabel('Number of Mentioners')
plt.ylabel('Number of Mentioned Mentioners')
plt.xscale('log')
plt.yscale('log')

Plot the hidden network:

In [ ]:
G_mentioning_giant = max(nx.connected_component_subgraphs(G_mentioning), key=len)
pos_mentioning_giant = nx.spring_layout(D_mentioning, k=0.1, iterations=200)
plt.figure(figsize=(6, 6))
nx.draw(G_mentioning_giant, pos=pos_mentioning_giant, node_size=mentioning_inweight, node_color=politicians_color_mentioning)

Some basic graph statistics:

In [ ]:
n = nx.number_of_nodes(G_mentioning_giant)
print('Number of nodes:', n)
m = nx.number_of_edges(G_mentioning_giant)
print('Number of edges:', m)
d = nx.density(G_mentioning_giant)
print('Density:', d)
print('Average degree:', np.mean(mentioning_degree))
print('Degree correlation:', nx.degree_assortativity_coefficient(G_mentioning_giant))
c = np.mean(list(nx.clustering(G_mentioning_giant).values()))
print('Average clustering coefficient:', c)
l = nx.average_shortest_path_length(G_mentioning_giant)
print('Average shortest path length:', l)
print('Diameter:', nx.diameter(G_mentioning_giant))

A primer on small-world networks:

<a href=''>Small-world networks</a> are another paradigmatic class of graphs in network science. A network is a small world if it has two properties at the same time. First, it has the property of highly ordered networks that a node's neighbors are strongly connected among themselves (high average clustering coefficient). Second, it has the property of highly disordered networks that nodes have low degrees of seperation (low average shortest path length).

The small-world property is important because it quantifies the co-presence of two central characteristics of mature social systems.

In [ ]:
c_random = []
l_random = []
for i in range(1, 10):
    G_random = nx.erdos_renyi_graph(n, d)
    G_random_giant = max(nx.connected_component_subgraphs(G_random), key=len)
    c_random.append(np.mean(list(nx.clustering(G_random).values())))
    l_random.append(nx.average_shortest_path_length(G_random_giant))
c_random = np.mean(c_random)
l_random = np.mean(l_random)

In [ ]:
print('Observed average clustering coefficient:', round(c, 2))
print('Expected average clustering coefficient:', round(c_random, 2))
print('Observed average shortest path length:', round(l, 2))
print('Expected average shortest path length:', round(l_random, 2))
print('Small-world coefficient:', int(round((c/c_random)/(l/l_random), 0)))